In [9]:
import numpy as np
import csv
import igl
import math
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
from scipy.io import loadmat 
import os
import geopandas as gpd
import plotly.graph_objects as go

In [10]:
def readmitodata(fname):
    """
    Expect a filename of the form "mito_data???.mat" 
    where ??? is the mito number (left padded by 0 is necessary).
    
    argument:
    fname -- string, filename of the mito data
    
    returns:
    mito_dict -- dictionary, keys are the names of the variables in the matlab file
    The keys are:
    'mito' -- the mitochondrion binary shaoe (np.ndarray, has been padded with zeros)
    'vertices' -- the vertices of the meshed mitochondrion (np.ndarray)  
    'faces' -- the faces of the meshed mitochondrion (np.ndarray)
    'cristae_junctions' -- the vertices of the cristae junctions (np.ndarray) or empty list if none
    'min_coords' -- the minimum coordinates of the mitochondrion (np.ndarray) in the original volume    
    'mito_number' -- the number of the mitochondrion (int) from the cc3d.component_labeling function,
                        this is the same as the number in the filename
    
    """
    mito_dict = loadmat(fname)
    # remove matlab meta data
    del mito_dict['__header__']
    del mito_dict['__version__']
    del mito_dict['__globals__']
    mito_dict['mito_number'] = int(mito_dict['mito_number'])
    # inverted results in matlab....
    mito_dict['vertices'], mito_dict['faces'] = mito_dict['faces'], mito_dict['vertices']
    # count starts at 0
    mito_dict['faces'] -= 1
    return mito_dict

In [11]:
def shortest_path(vecs, faces, vs, vt):

    shortest_path = math.inf

    # get the indices and coordinates of the closest vertices to the source and target points
    ds = igl.signed_distance(vs, vecs, faces)
    svec = faces[ds[1], :]
    svec_coords = vecs[svec, :]
    dt = igl.signed_distance(vt, vecs, faces)
    tvec = faces[dt[1], :]
    tvec_coords = vecs[tvec, :]

    # find the shortest path between the source and target points
    dist_vs_vecs = np.linalg.norm(vs-svec_coords, axis=1)
    # print(dist_vs_vecs)
    dist_vt_vecs = np.linalg.norm(vt-tvec_coords, axis=1)
    # print(dist_vt_vecs)
    dist_vecs = igl.exact_geodesic(vecs, faces, svec, tvec)
    dist = dist_vs_vecs + dist_vecs + dist_vt_vecs
    shortest_path = np.min(dist)
    
    return shortest_path

def pair_distance_mesh(vecs, faces, samples):
    npts = np.shape(samples)[0]
    dist = []
    for i in range(npts):
        for j in range(i+1, npts):
            # reshape to (1,3) array
            vs = samples[i].reshape(1,3)
            vt = samples[j].reshape(1,3)
            d = shortest_path(vecs, faces, vs, vt)
            dist.append(d)
    return dist

def mesh_area(vecs, faces):
    double_areas = igl.doublearea(vecs, faces)
    surface_area = np.sum(double_areas) / 2.0
    return surface_area

def ripleyK_mesh(vecs, faces, data, radii):
    K = np.zeros_like(radii)
    area = mesh_area(vecs, faces)
    dists = pair_distance_mesh(vecs, faces, data)
    pair_num = len(dists)
    intensity = pair_num / area
    for i in range(len(radii)):
        K[i] = np.sum(dists < radii[i])
    K = K / intensity
    return K

In [12]:
current_dir = os.getcwd()
folder_path = os.path.join(current_dir, "mito_data")
path = os.path.join(folder_path, "mito_data002.mat")
mito = readmitodata(path)  

In [13]:
print(mito.keys())
vertices = np.array(mito['vertices'], dtype=np.float64)
faces = np.array(mito['faces'], dtype=np.int32)
cj = np.array(mito['cristae_junction'], dtype=np.float64)

print(cj.shape)
print(vertices.shape)
print(faces.shape)

print(vertices.min(), vertices.max())
print(faces.min(), faces.max())

dict_keys(['cristae_junction', 'faces', 'min_coords', 'mito', 'mito_number', 'vertices'])
(3, 6)
(5990, 3)
(11976, 3)
1.505 44.495
0 5989


In [14]:
points = cj.T
print(points.shape)

x, y, z = vertices.T
i, j, k = faces.T

mesh = go.Mesh3d(x=x, y=y, z=z, i=i, j=j, k=k, color='lightpink', opacity=0.50)
fig = go.Figure(data=[mesh])
fig.add_trace(go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=10, color='blue')))
fig.show()

(6, 3)


In [15]:
#  Computes signed distance to a mesh

#  Inputs:
#    P  #P by 3 list of query point positions
#    V  #V by 3 list of vertex positions
#    F  #F by ss list of triangle indices, ss should be 3 unless sign_type ==
#      SIGNED_DISTANCE_TYPE_UNSIGNED
#    sign_type  method for computing distance _sign_ S
#    lower_bound  lower bound of distances needed {std::numeric_limits::min}
#    upper_bound  lower bound of distances needed {std::numeric_limits::max}
#  Outputs:
#    S  #P list of smallest signed distances
#    I  #P list of facet indices corresponding to smallest distances
#    C  #P by 3 list of closest points
#    N  #P by 3 list of closest normals (only set if
#      sign_type=SIGNED_DISTANCE_TYPE_PSEUDONORMAL)

if points != []:
    signed_dist = igl.signed_distance(points, vertices, faces)
    print(signed_dist)

/tmp/ipykernel_6985/3795998529.py:18: DeprecationWarning:

elementwise comparison failed; this will raise an error in the future.



(array([ 6.06506458,  1.485     ,  7.43673483,  2.47840927,  2.87663432,
       31.48642922]), array([3610, 7004, 4187, 7334, 7613, 8534], dtype=int32), array([[ 4.50166667, 16.49833333, 16.50166667],
       [ 1.505     , 13.        , 28.        ],
       [ 7.        , 20.495     , 18.        ],
       [ 7.7525    , 25.2475    , 29.        ],
       [ 7.        , 25.495     , 31.        ],
       [17.        , 22.495     , 34.        ]]))


In [16]:
vs = points[0].reshape(1,3)
vt = points[1].reshape(1,3)

print(vs)
print(vt)


[[ 1. 20. 13.]]
[[ 2. 13. 28.]]


In [17]:
vecs = vertices
faces = faces

shortest_path = math.inf

# get the indices and coordinates of the closest vertices to the source and target points
ds = igl.signed_distance(vs, vecs, faces)
svec = faces[ds[1], :] #find the 3 cloesest vertices
svec_coords = vecs[svec, :] #Find the coordinates for the closest vertices
dt = igl.signed_distance(vt, vecs, faces)
tvec = faces[dt[1], :]
tvec_coords = vecs[tvec, :]

"""
# find the shortest path between the source and target points
dist_vs_vecs = np.linalg.norm(vs-svec_coords, axis=1)
print(dist_vs_vecs)
dist_vt_vecs = np.linalg.norm(vt-tvec_coords, axis=1)
print(dist_vt_vecs)
dist_vecs = igl.exact_geodesic(vecs, faces, svec, tvec) # this is not working crashing the kernel
dist = dist_vs_vecs + dist_vecs + dist_vt_vecs
shortest_path = np.min(dist)
"""

: 

: 